In [57]:
from PIL import Image, ImageFilter
import glob
import os
import numpy as np
import cv2
from random import *
import shutil

In [58]:
no_blur_path = 'StreetView/train/no_blur'
no_blur_label_path ='StreetView/train/no_blur_label'
blur_label_path = 'StreetView/train/blur_label'
blur_path = 'StreetView/train/blur'
folder= 'part1'

In [59]:
#img = cv2.imread('temp/0.jpg')
#print(img.shape)
os.path.split('StreetView/train/blur/01.jpg')[1]

'01.jpg'

In [60]:
def blurizing_creating_blur_label(blur_path,blur_label_path,filename):
    #image_blur = image.copy()
    image_blur =Image.open(filename)
    img = cv2.imread(filename)
    width, height = image_blur.size
    dic ={'1':{'h1':int( height / 4),'h2':800,'w1':900,'w2':1100,'left':900},
      '2':{'h1':int( height / 3),'h2':600,'w1':100,'w2':500,'left':100},
       '3':{'h1':int( height / 4),'h2':700,'w1':500,'w2':800,'left':500},
       '4':{'h1':int( height / 4),'h2':700,'w1':700,'w2':1000,'left':700},
       '5':{'h1':int( height / 4),'h2':1000,'w1':100,'w2':400,'left':100},
       '6':{'h1':int(2* height / 4),'h2':1000,'w1':100,'w2':400,'left':100},
       '7':{'h1':int(height / 4),'h2':900,'w1':100,'w2':400,'left':100},
       '8':{'h1':int(height / 5),'h2':800,'w1':800,'w2':1100,'left':800}}
    # Cropping an image
    x = randint(1,8)
    cropped_image = img[dic[str(x)]['h1']:dic[str(x)]['h2'], dic[str(x)]['w1']:dic[str(x)]['w2']]
    cropped_image=cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    blurred_image = Image.fromarray(cropped_image).filter(ImageFilter.GaussianBlur(10))
    image_blur.paste(blurred_image, (dic[str(x)]['left'], dic[str(x)]['h1']))
    image_blur = image_blur.resize((224,224))
    image_blur.save(blur_path+'/'+ os.path.split(filename)[1])
    #image_blur.save('{}{}{}'.format(blur_path,'/',os.path.split(filename)[1]))
    #label for blureed image 
    label_img = np.zeros((height, width, 3), dtype = "uint8")
    label_img = Image.fromarray(label_img)
    w,h = Image.fromarray(cropped_image).size
    whiteFrame = 255 * np.ones((h,w,3), np.uint8)
    whiteFrame = Image.fromarray(whiteFrame)
    label_img.paste(whiteFrame,(dic[str(x)]['left'], dic[str(x)]['h1']))
    label_img = label_img.resize((224,224))
    label_img.save('{}{}{}'.format(blur_label_path,'/',os.path.split(filename)[1]))
    

In [61]:
def resize_images_creating_label(new_path,no_blur_label_path,blur_path,blur_label_path,folder):
    temp = 'temp'
    if not os.path.exists(new_path ):
        os.makedirs(new_path )
    if not os.path.exists(no_blur_label_path):
        os.makedirs(no_blur_label_path)
    if not os.path.exists(temp):
        os.makedirs(temp)
    if not os.path.exists(blur_label_path):
        os.makedirs(blur_label_path)
    if not os.path.exists(blur_path):
        os.makedirs(blur_path)
    #starting processing images
    images_list=os.listdir(folder)
    i=0
    for item in images_list[:1000]:
        image = Image.open(folder+"/"+item)   
        image.save(temp +'/' + str(i)+'.jpg', 'JPEG', quality=90) 
        i+=1
    for filename in glob.glob(temp+'/' +'*.jpg'): #path of raw images
        blurizing_creating_blur_label(blur_path,blur_label_path,filename)
        image = Image.open(filename)
        width, height = image.size
        image = image.resize((224,224))
        # save resized images to new folder with existing filename
        image.save('{}{}{}'.format(no_blur_path,'/',os.path.split(filename)[1]))
        label_img = np.zeros((height, width, 3), dtype = "uint8")
        label_img = Image.fromarray(label_img)
        label_img = label_img.resize((224,224))
        label_img.save('{}{}{}'.format(no_blur_label_path,'/',os.path.split(filename)[1]))
    shutil.rmtree(temp)
    

In [62]:
resize_images_creating_label(no_blur_path,no_blur_label_path,blur_path,blur_label_path,folder)